In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time

# File and sheet details
file_path = 'TTP_Analysis_Report.xlsx'
sheet_name = 'TTP Validation_result'

# Load the specified sheet
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Add 'TTP Created Date' column if it doesn't exist
if 'TTP Created Date' not in df.columns:
    df['TTP Created Date'] = ''

# Generate MITRE ATT&CK URL from TTP
def generate_url(technique):
    if pd.isna(technique):
        return None
    technique = str(technique).strip()
    if '.' in technique:
        base, sub = technique.split('.')
        return f'https://attack.mitre.org/techniques/{base}/{sub}/'
    else:
        return f'https://attack.mitre.org/techniques/{technique}/'

# Extract "Created" date from the page
def extract_created_date(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code != 200:
            return None
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text(separator='\n')
        match = re.search(r'Created:\s+(\d{1,2} \w+ \d{4})', text)
        if match:
            return match.group(1)
    except Exception as e:
        print(f"Error fetching {url}: {e}")
    return None

# Loop through rows and populate date
for index, row in df.iterrows():
    ttp = row['TTP']
    url = generate_url(ttp)
    if url:
        print(f"Processing {ttp} => {url}")
        date = extract_created_date(url)
        df.at[index, 'TTP Created Date'] = date if date else 'Not found'
        time.sleep(1)

# Write the result to a new Excel file with the same sheet name
with pd.ExcelWriter('TTP_Analysis_Report_Updated.xlsx', engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name=sheet_name, index=False)

print("✅ Done. Updated file saved as 'TTP_Analysis_Report_Updated.xlsx'")


Processing T1218.005 => https://attack.mitre.org/techniques/T1218/005/
Processing T1059.002 => https://attack.mitre.org/techniques/T1059/002/
Processing T1053 => https://attack.mitre.org/techniques/T1053/
Processing T1127.001 => https://attack.mitre.org/techniques/T1127/001/
Processing T1590.001 => https://attack.mitre.org/techniques/T1590/001/
Processing T1218.002 => https://attack.mitre.org/techniques/T1218/002/
Processing T1218.007 => https://attack.mitre.org/techniques/T1218/007/
Processing T1548.003 => https://attack.mitre.org/techniques/T1548/003/
Processing T1071.001 => https://attack.mitre.org/techniques/T1071/001/
Processing T1012 => https://attack.mitre.org/techniques/T1012/
Processing T1505.003 => https://attack.mitre.org/techniques/T1505/003/
Processing T1030 => https://attack.mitre.org/techniques/T1030/
Processing T1547.002 => https://attack.mitre.org/techniques/T1547/002/
Processing T1055 => https://attack.mitre.org/techniques/T1055/
Processing T1036.004 => https://attack

In [3]:
pip install pandas openpyxl beautifulsoup4 requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
